In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [6]:
df.sample(6)

,age,gender,fever,cough,city,has_covid
73,34,Male,98.0,Strong,Kolkata,Yes
53,83,Male,98.0,Mild,Delhi,Yes
54,60,Female,99.0,Mild,Mumbai,Yes
92,82,Female,102.0,Strong,Kolkata,No
50,19,Male,101.0,Mild,Delhi,Yes
8,19,Female,100.0,Strong,Bangalore,No


In [8]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [9]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [10]:
df['fever'].value_counts()

98.0     17
101.0    17
104.0    14
100.0    13
102.0    10
99.0     10
103.0     9
Name: fever, dtype: int64

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [12]:
X_train

,age,gender,fever,cough,city
50,19,Male,101.0,Mild,Delhi
99,10,Female,98.0,Strong,Kolkata
86,25,Male,104.0,Mild,Bangalore
46,19,Female,101.0,Mild,Mumbai
91,38,Male,NaN,Mild,Delhi
...,...,...,...,...,...
19,42,Female,NaN,Strong,Bangalore
74,34,Female,104.0,Strong,Delhi
94,79,Male,NaN,Strong,Kolkata
11,65,Female,98.0,Mild,Mumbai


In [15]:
y_test

57     No
62    Yes
26    Yes
92     No
70     No
63     No
43    Yes
35     No
73    Yes
77     No
31     No
21    Yes
5     Yes
45     No
17     No
36     No
49     No
2      No
27     No
93    Yes
Name: has_covid, dtype: object

## 1. Aam Zindagi

In [23]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [24]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [25]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [26]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [27]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Mentos Zindagi

In [28]:
from sklearn.compose import ColumnTransformer

In [30]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [36]:
transformer.fit_transform(X_train).shape

(80, 7)

In [35]:
X_train

,age,gender,fever,cough,city
50,19,Male,101.0,Mild,Delhi
99,10,Female,98.0,Strong,Kolkata
86,25,Male,104.0,Mild,Bangalore
46,19,Female,101.0,Mild,Mumbai
91,38,Male,NaN,Mild,Delhi
...,...,...,...,...,...
19,42,Female,NaN,Strong,Bangalore
74,34,Female,104.0,Strong,Delhi
94,79,Male,NaN,Strong,Kolkata
11,65,Female,98.0,Mild,Mumbai


In [32]:
transformer.transform(X_test).shape

(20, 7)